In [1]:
### Mixture density network
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow_probability as tfp
tfd = tfp.distributions
tfpl = tfp.layers
tfk = tf.keras
tfkl = tf.keras.layers

In [2]:
currency = pd.read_excel('C:/Users/lenovo/Desktop/EURUSD.xlsx') # read the excel file
currency

,Date,Open,High,Low,Close
0,2022-06-14 12:36:00,1.0420,1.0420,1.0419,1.0419
1,2022-06-14 12:35:00,1.0421,1.0422,1.0419,1.0419
2,2022-06-14 12:34:00,1.0423,1.0423,1.0421,1.0421
3,2022-06-14 12:33:00,1.0422,1.0423,1.0421,1.0423
4,2022-06-14 12:32:00,1.0422,1.0422,1.0420,1.0422
...,...,...,...,...,...
4109,2022-06-09 17:04:00,1.0617,1.0618,1.0617,1.0618
4110,2022-06-09 17:03:00,1.0617,1.0617,1.0617,1.0617
4111,2022-06-09 17:02:00,1.0617,1.0617,1.0617,1.0617
4112,2022-06-09 17:01:00,1.0617,1.0617,1.0616,1.0617


In [3]:
# use new column 'return' instead of columns 'Open' and 'Close'

# add new column 'return'
currency['return'] = np.log(currency['Close']) - np.log(currency['Open'])
# delete columns 'Open' and 'Close'
del currency['Open'], currency['Close']
currency

,Date,High,Low,return
0,2022-06-14 12:36:00,1.0420,1.0419,-0.000096
1,2022-06-14 12:35:00,1.0422,1.0419,-0.000192
2,2022-06-14 12:34:00,1.0423,1.0421,-0.000192
3,2022-06-14 12:33:00,1.0423,1.0421,0.000096
4,2022-06-14 12:32:00,1.0422,1.0420,0.000000
...,...,...,...,...
4109,2022-06-09 17:04:00,1.0618,1.0617,0.000094
4110,2022-06-09 17:03:00,1.0617,1.0617,0.000000
4111,2022-06-09 17:02:00,1.0617,1.0617,0.000000
4112,2022-06-09 17:01:00,1.0617,1.0616,0.000000


In [39]:
cur_r= currency[['Date','return']]
cur_r.head(20)

,Date,return
0,2022-06-14 12:36:00,-0.000096
1,2022-06-14 12:35:00,-0.000192
2,2022-06-14 12:34:00,-0.000192
3,2022-06-14 12:33:00,0.000096
4,2022-06-14 12:32:00,0.000000
5,2022-06-14 12:31:00,-0.000096
6,2022-06-14 12:30:00,0.000000
7,2022-06-14 12:29:00,0.000000
8,2022-06-14 12:28:00,0.000288
9,2022-06-14 12:27:00,0.000288


In [5]:
#split train set (use data before June 14 as train set)
window = 5
x_train, y_train = [], []

for i in range(757,4114-5):
    x_train.append(cur_r.loc[i + 1 : i + window, 'return'])
    y_train.append(cur_r.loc[i, 'return'])

#convert the list to ndarray
x_train = np.array(x_train)
y_train = np.array(y_train)

In [6]:
# split test set (use data on June 14 as test set)
x_test, y_test = [], []

for i in range(752):
    x_test.append(cur_r.loc[i + 1 : i + window, 'return'])
    y_test.append(cur_r.loc[i, 'return'])

#convert the list to ndarray
x_test = np.array(x_test)
y_test = np.array(y_test)

In [7]:
# build the model
event_shape = [1]
num_components = 6  # normal mixture with 5 components
params_size = tfp.layers.MixtureNormal.params_size(num_components, event_shape)
model = tfk.Sequential([
    tfkl.Dense(10, activation='relu'),
    tfkl.Dense(12, activation='relu'),
    tfkl.Dense(10, activation='relu'),
    tfkl.Dense(params_size, activation=None),
    tfpl.MixtureNormal(num_components, event_shape)
])

In [40]:
# fit
model.compile(optimizer="adam",
              loss=lambda y, model: -model.log_prob(y))
model.fit(x_train, y_train, batch_size=64, epochs=200)

Epoch 1/200
53/53 [==============================] - 1s 1ms/step - loss: 13.4639
Epoch 2/200
53/53 [==============================] - 0s 1ms/step - loss: 5.0645
Epoch 3/200
53/53 [==============================] - 0s 1ms/step - loss: 4.0150
Epoch 4/200
53/53 [==============================] - 0s 1ms/step - loss: 2.9962
Epoch 5/200
53/53 [==============================] - 0s 1ms/step - loss: 1.9820
Epoch 6/200
53/53 [==============================] - 0s 1ms/step - loss: 0.9720
Epoch 7/200
53/53 [==============================] - 0s 1ms/step - loss: -0.0329
Epoch 8/200
53/53 [==============================] - 0s 1ms/step - loss: -1.0249
Epoch 9/200
53/53 [==============================] - 0s 1ms/step - loss: -1.9791
Epoch 10/200
53/53 [==============================] - 0s 1ms/step - loss: -2.8423
Epoch 11/200
53/53 [==============================] - 0s 1ms/step - loss: -3.5476
Epoch 12/200
53/53 [==============================] - 0s 1ms/step - loss: -4.0691
Epoch 13/200
53/53 [==========

53/53 [==============================] - 0s 1ms/step - loss: -5.7952
Epoch 102/200
53/53 [==============================] - 0s 1ms/step - loss: -4.6341
Epoch 103/200
53/53 [==============================] - 0s 1ms/step - loss: -5.4151
Epoch 104/200
53/53 [==============================] - 0s 1ms/step - loss: -5.9175
Epoch 105/200
53/53 [==============================] - 0s 1ms/step - loss: -6.0694
Epoch 106/200
53/53 [==============================] - 0s 1ms/step - loss: -5.8968
Epoch 107/200
53/53 [==============================] - 0s 1ms/step - loss: -5.4313
Epoch 108/200
53/53 [==============================] - 0s 1ms/step - loss: -6.1643
Epoch 109/200
53/53 [==============================] - 0s 1ms/step - loss: -5.9292
Epoch 110/200
53/53 [==============================] - 0s 1ms/step - loss: -4.5443
Epoch 111/200
53/53 [==============================] - 0s 1ms/step - loss: -6.0091
Epoch 112/200
53/53 [==============================] - 0s 1ms/step - loss: -6.1656
Epoch 113/200
53/5

53/53 [==============================] - 0s 1ms/step - loss: -5.8503
Epoch 200/200
53/53 [==============================] - 0s 1ms/step - loss: -6.0686


In [41]:
# This is an example
x_0 = np.array([[-1.91809725e-04,  0.00000000e+00,  0.00000000e+00,
        -9.58910678e-05,  2.87700793e-04]])
gm = model(x_0)

# get the mixture parameters:
gm_weights = gm.mixture_distribution.probs_parameter().numpy().flatten()
gm_means = gm.components_distribution.mean().numpy().flatten()
gm_vars = gm.components_distribution.variance().numpy().flatten()

print("Component weights: ", gm_weights)
print("Component means: ", gm_means)
print("Component variances: ", gm_vars)

print("Predicted mean: ", np.dot(gm_weights, gm_means))
print("Predicted st dev: ", np.sqrt(np.dot(gm_weights**2, gm_vars)))

Component weights:  [9.9987757e-01 6.8404549e-09 1.2237711e-04 8.7185237e-09 4.2257198e-09
 5.0645519e-09]
Component means:  [-4.7314912e-05  7.6555885e-02 -7.4853674e-02 -1.2809485e-01
  1.7308798e-01  1.3605346e-01]
Component variances:  [6.8553294e-07 2.1481035e-06 3.2352412e-07 8.9094283e-06 9.5239056e-06
 2.1233202e-05]
Predicted mean:  -5.646867e-05
Predicted st dev:  0.00082786777


In [42]:
# Use mean as prediction.Campare the real return with the predicted return.
# For each x in testing set, compute the corresponding E[y|x]
y_pred = []
for i in range(x_test.shape[0]):
    x = x_test[i]
    x = x.reshape(1,x.shape[0])
    gm = model(x)
    gm_weights = gm.mixture_distribution.probs_parameter().numpy().flatten()
    gm_means = gm.components_distribution.mean().numpy().flatten()
    y_mean = np.dot(gm_weights, gm_means)
    y_pred.append(y_mean)

In [43]:
y_pred = np.array(y_pred)
y_pred.shape

(752,)

In [44]:
valid_r = cur_r[:y_pred.shape[0]]
valid_r['pred_r'] = y_pred
valid_r

<ipython-input-44-8e820d4fd1c0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_r['pred_r'] = y_pred


,Date,return,pred_r
0,2022-06-14 12:36:00,-0.000096,-0.000025
1,2022-06-14 12:35:00,-0.000192,-0.000040
2,2022-06-14 12:34:00,-0.000192,-0.000043
3,2022-06-14 12:33:00,0.000096,-0.000058
4,2022-06-14 12:32:00,0.000000,-0.000070
...,...,...,...
747,2022-06-14 00:09:00,0.000000,-0.000044
748,2022-06-14 00:08:00,0.000096,-0.000050
749,2022-06-14 00:07:00,-0.000096,-0.000045
750,2022-06-14 00:06:00,-0.000096,-0.000044


In [46]:
# The results are not precise, but much better than CNN.
y_pred

array([-2.52365280e-05, -3.96277792e-05, -4.32588058e-05, -5.76008024e-05,
       -7.00862802e-05, -5.75752601e-05, -4.34293033e-05, -4.99842827e-05,
       -5.99807972e-05, -2.64104710e-05, -1.71138106e-06, -4.49097024e-05,
       -6.53238822e-05, -1.42910576e-05, -4.04710954e-05, -8.12319122e-05,
       -5.34843857e-05, -3.77392571e-05, -4.43094541e-05, -2.23294046e-05,
       -3.98963093e-05, -5.39129906e-05, -1.27931353e-05, -3.72568466e-05,
       -6.01265929e-05, -2.67395953e-05, -3.71140013e-05, -4.60609626e-05,
       -4.47521961e-05, -5.67563256e-05, -4.53298053e-05, -1.95885896e-05,
       -3.73663061e-05, -7.40714167e-05, -1.86247471e-05, -2.46101172e-05,
       -8.39864078e-05, -5.15400570e-05, -5.57013227e-05, -3.91619615e-05,
       -2.39145356e-05, -4.49323525e-05, -5.28216151e-05, -4.70285740e-05,
       -6.15167801e-06, -4.14337810e-05, -7.47270315e-05, -2.19763097e-05,
       -2.39460314e-05, -2.98530413e-05, -4.26812185e-05, -3.51242306e-05,
       -8.76492777e-06, -